In [81]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [82]:
import numpy as np
import pandas as pd

In [83]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [84]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [85]:
# Create database connection
engine = create_engine("sqlite:///hawaii.sqlite")

In [86]:
# reflect an existing database into a new model
Base = automap_base()

In [87]:
# reflect the tables
Base.prepare(engine, reflect=True)

# View the classes imported
Base.classes.keys()

['measurement', 'station']

In [88]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [89]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [90]:
# Display Measurement data in dictionary format
first_row = session.query(Measurement).first()
first_row.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x1fae56346a0>,
 'tobs': 65.0,
 'date': '2010-01-01',
 'id': 1,
 'prcp': 0.08,
 'station': 'USC00519397'}

In [112]:
prcp_test = session.query(Measurement).filter(Measurement.prcp == 0.08).all()
for row in prcp_test:
    print(f"Prcp: {row.prcp}")

Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08
Prcp: 0.08

# Exploratory Climate Analysis

In [118]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results
# Calculate the date 1 year ago from the last data point in the database
last_date = session.query(Measurement.prcp).order_by(Measurement.prcp.desc()).first()
print(f"The last date in the file is {last_date}.)

# ***FIX: NOT WORKING***

SyntaxError: EOL while scanning string literal (<ipython-input-118-85cf77747b8b>, line 4)

In [126]:
# Perform a query to retrieve the date and precipitation scores
date_prcp = session.query(Measurement.date, Measurement.prcp).all()

# Save the query results as a Pandas DataFrame and set the index to the date column
prcp_df = pd.DataFrame(date_prcp)
prcp_df.dtypes

date     object
prcp    float64
dtype: object

In [127]:
# Remove null values
prcp_clean = prcp_df.dropna()

# Sort the dataframe by date
prcp_by_date = prcp_clean.sort_values(by=['date'])

# Convert dates from strings to datetime
prcp_by_date.iloc[prcp_by_date.to_datetime(prcp_by_date.date).values.argsort()]
prcp_by_date.dtypes

# Keep only last 12 months worth of data
prcp_12mos = prcp_by_date(date > 2016-08-23)

# Use Pandas Plotting with Matplotlib plot the data

AttributeError: 'DataFrame' object has no attribute 'to_datetime'

In [93]:
# Use Pandas to calcualte the summary statistics for the precipitation data

In [94]:
# Display Station data in dictionary format
first_row_station = session.query(Station).first()
first_row_station.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x1fae563be48>,
 'longitude': -157.8168,
 'name': 'WAIKIKI 717.2, HI US',
 'id': 1,
 'elevation': 3.0,
 'latitude': 21.2716,
 'station': 'USC00519397'}

In [95]:
# Design a query to show how many stations are available in this dataset?


In [96]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.


In [97]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature of the most active station?


In [98]:
# Choose the station with the highest number of temperature observations. (use func.max or func.count)
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram with bins=12


## Bonus Challenge Assignment

In [99]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

[(62.0, 69.57142857142857, 74.0)]


In [100]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.


In [101]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)


In [102]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation



In [103]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

[(62.0, 69.15384615384616, 77.0)]

In [104]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip

# Use the start and end date to create a range of dates

# Stip off the year and save a list of %m-%d strings

# Loop through the list of %m-%d strings and calculate the normals for each date


In [105]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index


In [106]:
# Plot the daily normals as an area plot with `stacked=False`
